<a href="https://colab.research.google.com/github/Taaniya/LLM-compression-optimization/blob/main/Loading_LLM_T5_with_int8_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HuggingFace meets `bitsandbytes` for lighter models on GPU for inference

## Running T5-11b on Google Colab

 <center>
 <img src="https://s3.amazonaws.com/moonup/production/uploads/1659861207959-62441d1d9fdefb55a0b7d12c.png">
 </center>


You can run your own 8-bit model on any HuggingFace 🤗 model with just few lines of code. This notebook shows how to do it with a `T5` model that would usually require 12GB of GPU RAM.
Install the dependencies below first!


In [1]:
!pip install --quiet bitsandbytes
!pip install --quiet --upgrade transformers # Install latest version of transformers
!pip install --quiet --upgrade accelerate
!pip install --quiet sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers bitsandbytes accelerate

## Choose your model

Rerun this cell if you want to change the model!

In [3]:
model_name = "t5-3b-sharded" #@param ["t5-11b-sharded", "t5-3b-sharded"]

This model has sharded checkpoints, i.e., it involves a single folder with several files , where each file is a checkpoint containing the partial state dict i.e., the model weights.

We can access the model's its sharded checkpoint on its model card for it [here](https://huggingface.co/ybelkada/t5-3b-sharded/tree/main)

Accelerate is used to load large sharded models when the entire model cannot fit into RAM. More details of how accelerate enables this can be referred [here](https://huggingface.co/docs/accelerate/usage_guides/big_modeling) and [here](https://huggingface.co/docs/transformers/main/en/main_classes/model#large-model-loading).

## Use 8bit models with `t5-3b-sharded` 🤗

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# T5-3b and T5-11B are supported!
# We need sharded weights otherwise we get CPU OOM errors
model_id=f"ybelkada/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_8bit = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at ybelkada/t5-3b-sharded and are newly initialized: ['encoder.embed_tokens.weight', 'lm_head.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


By passing device_map="auto", we tell Accelerate to determine automatically where to put each layer of the model depending on the available resources - GPU RAM, CPU RAM or even disk.

We can inspect how the model was split across devices by looking at its hf_device_map attribute. In this case, I've loaded this model on a single GPU on colab.

In [10]:
model_8bit.hf_device_map

{'': 0}

Let's check the memory footprint of this model! 🪶

In [5]:
model_8bit.get_memory_footprint()

5300543488

We use quantization while loading the model for inference to reduce the memory footprint by passing argument `load_in_8bit=True` to load the model weights with lower precision - 8 bit integers.

Quantization is kind of a lossy compression of representing weights with data types at lower precision.

This is explained in detail in [this Huggingface blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) and how its enabled with help of bitsandbytes.

For `t5-3b` the int8 model is about ~2.9GB! whereas the original model has 11GB. For `t5-11b` the int8 model is about ~11GB vs 42GB for the original model.
Now let's generate and see the qualitative results of the 8bit model!

In [7]:
max_new_tokens = 50

input_ids = tokenizer(
    "translate English to German: Hello my name is Younes and I am a Machine Learning Engineer at Hugging Face", return_tensors="pt"
).input_ids

outputs = model_8bit.generate(input_ids, max_new_tokens=max_new_tokens)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Hallo mein Name ist Younes und ich bin ein Machine Learning Ingenieur bei Hugging Face


#### References

* What is sharding - https://docs.graphcore.ai/projects/tf-model-parallelism/en/latest/sharding.html
* Large model loading - https://huggingface.co/docs/transformers/main/en/main_classes/model#large-model-loading
* Handling big models for inference - https://huggingface.co/docs/accelerate/usage_guides/big_modeling
* https://huggingface.co/blog/hf-bitsandbytes-integration
* https://huggingface.co/blog/4bit-transformers-bitsandbytes
* Quantization - https://huggingface.co/docs/accelerate/usage_guides/quantization